In [324]:
%matplotlib inline
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from pandas import set_option
set_option("display.max_rows", 20)

LARGE_FIGSIZE = (12, 8)

In [17]:
cd data

/Users/esteban/unm/MachineLearning/Project1/data


In [18]:
ls

testing.txt     training.txt    validation.txt


In [525]:
training = pd.read_table("training.txt", sep=",", names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                                                  "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                                                  "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                                                  "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                                                  "veil-color","ring-number","ring-type","spore-print-color",
                                                  "population","habitat"])
training

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,y,e,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
1,e,x,f,g,f,n,f,c,n,n,...,s,w,w,p,w,o,p,k,v,u
2,p,f,s,n,t,p,f,c,n,p,...,s,w,w,p,w,o,p,n,s,u
3,p,x,y,g,f,f,f,c,b,g,...,k,b,n,p,w,o,l,h,y,p
4,e,x,y,g,t,n,f,c,b,n,...,s,p,g,p,w,o,p,k,y,d
5,e,x,f,w,t,a,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
6,e,b,f,g,f,n,f,w,b,p,...,k,w,w,p,w,t,p,w,s,g
7,e,x,f,n,t,n,f,c,b,n,...,s,p,p,p,w,o,p,n,v,d
8,e,k,f,c,f,n,f,w,n,w,...,s,w,n,p,w,o,e,w,v,l
9,e,s,f,g,f,n,f,c,n,k,...,s,w,w,p,w,o,p,k,v,u


In [570]:
import math

class Node:
    decision_attribute = ""
    
    
    def __init__(self):
        self.decision_attribute = ""
        self.branches = {}
        
    def add_branch(self,attribute_value,node):
        self.branches[attribute_value] = node
    
    def get_branches(self):
        return self.branches

class Leaf:
    label = ""
    
positive_value = ""
negative_value = ""
evaluation_criteria = ""

attributes_master = ["cap-shape","cap-surface","cap-color","bruises","oder",
                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                "veil-color","ring-number","ring-type","spore-print-color",
                "population","habitat"]

valid_values = {"label": ["p","e"],
                "cap-shape": ["b","c","x","f","k","s"],
                "cap-surface": ["f","g","y","s"],
                "cap-color": ["n","b","c","g","r","p","u","e","w","y"],
                "bruises": ["t","f"],
                "oder": ["a","l","c","y","f","m","n","p","s"],
                "gill-attachment": ["a","d","f","n"],
                "gill-spacing": ["c","w","d"],
                "gill-size": ["b","n"],
                "gill-color": ["k","n","b","h","g","r","o","p","u","e","w","y"],
                "stalk-shape": ["e","t"],
                "stalk-root": ["b","c","u","e","z","r","?"],
                "stalk-surface-above-ring": ["f","y","k","s"],
                "stalk-surface-below-ring": ["f","y","k","s"],
                "stalk-color-above-ring": ["n","b","c","g","o","p","e","w","y"],
                "stalk-color-below-ring": ["n","b","c","g","o","p","e","w","y"],
                "veil-type": ["p","u"],
                "veil-color": ["n","o","w","y"],
                "ring-number": ["n","o","t"],
                "ring-type": ["c","e","f","l","n","p","s","z"],
                "spore-print-color": ["k","n","b","h","r","o","u","w","y"],
                "population": ["a","c","n","s","v","y"],
                "play-tennis": ["y","n"],
                "habitat": ["g","l","m","p","u","w","d"],
                "outlook": ["s","o", "r"],
                "temperature": ["h","m","c"],
                "wind": ["w","s"],
                "humidity": ["h","n"]}
    
def all_one_value(examples, attribute, attribute_value):
    #test that all examples have the same value, examples are a Pandas Dataframe
    filtered_examples = examples[examples[attribute] == attribute_value]
    return len(filtered_examples.index) == len(examples.index)

def extract_range_of_values(examples, attribute):
    #return pd.unique(examples[attribute])
    return valid_values[attribute]

def extract_most_common_value(examples, attribute):
    #print("most common for ", attribute)
    values = extract_range_of_values(examples,attribute)
    most_common = ""
    highest_count = 0
    for value in values:
        filtered = examples[examples[attribute] == value]
        if len(filtered.index) > highest_count:
            highest_count = len(filtered.index)
            most_common = value  
    return most_common

#def calculate_entropy_old(examples, attribute):
    #determine possible values for attribute
    #possible_values = extract_range_of_values(examples, attribute)
    #entropy = 0.0
    #total = len(examples.index)
    #for v in possible_values:
        #filtered_examples = examples[examples[attribute] == v]
        #total = len(filtered_examples.index)
        #positive_filtered_examples = filtered_examples[filtered_examples["label"] == positive_value]
        #negative_filtered_examples = filtered_examples[filtered_examples["label"] == negative_value]
        #num_positive = len(positive_filtered_examples.index)
        #calc_positive = 0.0
        #if num_positive != 0:
            #calc_positive = -(num_positive/total)*math.log((num_positive/total),2)
            
        
        #num_negative = len(negative_filtered_examples.index)
        #calc_negative = 0.0
        #if num_negative != 0:
            #calc_negative = - (num_negative/total)*math.log((num_negative/total),2)
        
        
        #temp_calc = calc_positive + calc_negative
        #entropy = entropy + temp_calc
    
    #return entropy

def calculate_entropy(examples,target_attribute):
    entropy = 0.0
    total = len(examples.index)
    positive_filtered_examples = examples[examples[target_attribute] == positive_value]
    negative_filtered_examples = examples[examples[target_attribute] == negative_value]
    
    num_positive = len(positive_filtered_examples.index)
    calc_positive = 0.0
    if num_positive != 0:
        calc_positive = -(num_positive/total)*math.log((num_positive/total),2)
            
        
    num_negative = len(negative_filtered_examples.index)
    calc_negative = 0.0
    if num_negative != 0:
        calc_negative = - (num_negative/total)*math.log((num_negative/total),2)
        
        
    return calc_positive + calc_negative
   

def calculate_misclassification_error(examples, target_attribute):
    error = 0.0
    total = len(examples.index)
    if total == 0:
        return 1.0
    positive_filtered_examples = examples[examples[target_attribute] == positive_value]
    negative_filtered_examples = examples[examples[target_attribute] == negative_value]
    
    num_positive = len(positive_filtered_examples.index)
    p_positive = (num_positive/total)
                
    num_negative = len(negative_filtered_examples.index)
    p_negative = (num_negative/total)
        
        
    
    error = 1.0 - max([p_positive,p_negative])
    return error
    
    
def determine_best_attribute_entropy(examples, target_attribute, attributes):
    best_attribute = ""
    max_information_gain = 0.0
    #print(attributes)
    #find the entropy S
    entropy_s = calculate_entropy(examples,target_attribute)
    size_of_s = len(examples.index)
    
    #count = 1
    #loop through attributes and calculate the gain
    for attribute in attributes:
        values = extract_range_of_values(examples, attribute)
        information_gain = entropy_s
        for value in values:
            examples_v = examples[examples[attribute] == value]
            entropy_v = calculate_entropy(examples_v,target_attribute)
            size_of_examples_v = len(examples_v.index)
            weight = (size_of_examples_v/size_of_s)
            information_gain = information_gain - weight*entropy_v
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_attribute = attribute
        #print(count,attribute, ": ", information_gain)
        #count = count + 1
       
    return best_attribute

def determine_best_attribute_misclassification_error(examples, target_attribute, attributes):
    best_attribute = ""
    max_information_gain = 0.0
    #print(attributes)
    #find the entropy S
    error_s = calculate_misclassification_error(examples,target_attribute)
    size_of_s = len(examples.index)
    
    #loop through attributes and calculate the gain
    for attribute in attributes:
        values = extract_range_of_values(examples, attribute)
        information_gain = error_s
        for value in values:
            examples_v = examples[examples[attribute] == value]
            error_v = calculate_misclassification_error(examples_v,target_attribute)
            size_of_examples_v = len(examples_v.index)
            weight = (size_of_examples_v/size_of_s)
            information_gain = information_gain - weight*error_v
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_attribute = attribute
       
    return best_attribute

def id3(examples, target_attribute, attribute_list):
    root = Node()
    if(all_one_value(examples, target_attribute, positive_value)):
        leaf = Leaf()
        leaf.label = positive_value
        return leaf
    if(all_one_value(examples, target_attribute, negative_value)):
        leaf = Leaf()
        leaf.label = negative_value
        return leaf
    if(len(attributes) == 0):
        leaf = Leaf()
        leaf.label = extract_most_common_value(examples,target_attribute)
        return leaf
    if evaluation_criteria == 'entropy':
        a = determine_best_attribute_entropy(examples, target_attribute, attribute_list)
    else:
        a = determine_best_attribute_misclassification_error(examples, target_attribute, attribute_list)
    root.decision_attribute = a
    values = extract_range_of_values(examples, a)
    #print(a,values)
    for value in values:
        root.branches[value] = ""
        examples_v = examples[examples[a] == value]
        size_of_examples_v = len(examples_v.index)
        if size_of_examples_v == 0:
            #root.label =  extract_most_common_value(examples,"label") 
            leaf = Leaf()
            leaf.label = extract_most_common_value(examples,target_attribute)
            root.add_branch(value,leaf)
            #print("adding branch ", value, " for ", a)
            #root.branches[value] = leaf
        else:
            if a in attribute_list:
                attribute_list.remove(a)
                #print("removing ", a)
            #root.branches[value] = id3(examples_v,target_attribute,attribute_list)
            node = id3(examples_v,target_attribute,attribute_list)
            root.add_branch(value, node)
            #print("adding branch ", value, " for ", a)
    return root


def split_child(attribute_values, node):
    return node.get_branches()[attribute_values[node.decision_attribute]]

def classify(attribute_values, node):
    if type(node) is Leaf:
        return node.label
    else:
        return classify(attribute_values, split_child(attribute_values,node))
    
    
    

In [466]:
play_data = pd.read_table("play_tennis.txt", sep=",", names=["play-tennis","outlook","temperature","humidity","wind"])
play_data

,play-tennis,outlook,temperature,humidity,wind
0,n,s,h,h,w
1,n,s,h,h,s
2,y,o,h,h,w
3,y,r,m,h,w
4,y,r,c,n,w
5,n,r,c,n,s
6,y,o,c,n,s
7,n,s,m,h,w
8,y,s,c,n,w
9,y,r,m,n,w


In [471]:
positive_value = "y"
negative_value = "n"
print(calculate_entropy2(play_data,"play-tennis"))

0.9402859586706309


In [470]:
positive_value = "y"
negative_value = "n"
humidity_high_play_data = play_data[play_data["humidity"] == 'h']
print(calculate_entropy2(humidity_high_play_data,"play-tennis"))

0.9852281360342516


In [469]:
positive_value = "y"
negative_value = "n"
best = determine_best_attribute_entropy(play_data,"play-tennis",["outlook","temperature","humidity","wind"])
print(best)

outlook


In [95]:
common = extract_most_common_value(play_data,  "wind") 
print(common)

w


In [467]:
positive_value = "y"
negative_value = "n"
root = id3(play_data,"play-tennis",["outlook","temperature","humidity","wind"])
print("                                             ",root.decision_attribute)
print("                               ",root.get_branches().keys())
print("                   ",root.get_branches()["r"].decision_attribute,
      "                    " ,root.get_branches()["s"].decision_attribute,
      "           ", root.get_branches()["o"].label)
print("     ",root.get_branches()["r"].get_branches().keys(), "       ", 
      root.get_branches()["s"].get_branches().keys())
print("                ",root.get_branches()["r"].get_branches()["w"].label,
      "     ", root.get_branches()["r"].get_branches()["s"].label,
      "                   ", root.get_branches()["s"].get_branches()["n"].label,
      "    ",root.get_branches()["s"].get_branches()["h"].label)



                                              outlook
                                dict_keys(['r', 's', 'o'])
                    wind                      humidity             y
      dict_keys(['w', 's'])         dict_keys(['n', 'h'])
                 y       n                     y      n


In [569]:
attributes = ["cap-shape","cap-surface","cap-color","bruises","oder",
                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                "veil-color","ring-number","ring-type","spore-print-color",
                "population","habitat"]
positive_value = "e"
negative_value = "p"
evaluation_criteria = "entropy"
id3_root = id3(training,"label",attributes)

1 cap-shape :  0.04990703082442968
2 cap-surface :  0.030069739120635375
3 cap-color :  0.03626086676772894
4 bruises :  0.18297550359272463
5 oder :  0.903102376238315
6 gill-attachment :  0.013361169275671991
7 gill-spacing :  0.10208691232789938
8 gill-size :  0.22045817823464509
9 gill-color :  0.4108443574501191
10 stalk-shape :  0.00797122955220364
11 stalk-root :  0.13124235688758285
12 stalk-surface-above-ring :  0.2927394647314604
13 stalk-surface-below-ring :  0.2649159254354383
14 stalk-color-above-ring :  0.26556565733836324
15 stalk-color-below-ring :  0.2518508472431633
16 veil-type :  0.0
17 veil-color :  0.024159323499753582
18 ring-number :  0.04002169138836216
19 ring-type :  0.314128079817105
20 spore-print-color :  0.49336770849823947
21 population :  0.1961173596118862
22 habitat :  0.15465988657959434
1 cap-shape :  0.05543608712578327
2 cap-surface :  0.01761403445673851
3 cap-color :  0.09555854877785036
4 bruises :  0.00031764353891210184
5 gill-attachment :  0

In [565]:
print(id3_root.decision_attribute)
print(id3_root.get_branches().keys())
print(id3_root.get_branches()["l"].label)
print(id3_root.get_branches()["n"].decision_attribute)
print(id3_root.get_branches()["n"].get_branches().keys())
print(id3_root.get_branches()["s"].label)
print(id3_root.get_branches()["f"].label)
print(id3_root.get_branches()["y"].label)
print(id3_root.get_branches()["m"].label)
print(id3_root.get_branches()["a"].label)
print(id3_root.get_branches()["c"].label)
print(id3_root.get_branches()["p"].label)
print(id3_root.get_branches()["n"].decision_attribute)
print(id3_root.get_branches()["n"].get_branches()["n"].label)
print(id3_root.get_branches()["n"].get_branches()["w"].decision_attribute)
print(id3_root.get_branches()["n"].get_branches()["w"].get_branches().keys())
print(id3_root.get_branches()["n"].get_branches()["h"].label)
print(id3_root.get_branches()["n"].get_branches()["b"].label)
print(id3_root.get_branches()["n"].get_branches()["k"].label)
print(id3_root.get_branches()["n"].get_branches()["y"].label)
print(id3_root.get_branches()["n"].get_branches()["r"].label)
print(id3_root.get_branches()["n"].get_branches()["u"].label)
print(id3_root.get_branches()["n"].get_branches()["o"].label)

oder
dict_keys(['l', 'n', 's', 'f', 'y', 'm', 'a', 'c', 'p'])
e
spore-print-color
dict_keys(['n', 'w', 'h', 'b', 'k', 'y', 'r', 'u', 'o'])
p
p
p
p
e
p
p
spore-print-color
e
habitat
dict_keys(['l', 'w', 'u', 'm', 'd', 'g', 'p'])
e
e
e
e
p
e
e


In [497]:
testing_data = pd.read_table("testing.txt", sep=",", names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                                                  "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                                                  "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                                                  "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                                                  "veil-color","ring-number","ring-type","spore-print-color",
                                                  "population","habitat"])
testing_data


,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,n,f,n,a,c,b,o,...,s,o,o,p,n,o,p,y,v,l
1,e,f,y,n,t,n,f,c,b,w,...,s,g,w,p,w,o,p,n,v,d
2,p,f,y,g,f,f,f,c,b,g,...,k,n,n,p,w,o,l,h,v,d
3,e,x,s,w,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,m
4,e,f,y,u,f,n,f,c,n,h,...,f,w,w,p,w,o,f,h,v,d
5,e,k,f,c,f,n,f,w,n,w,...,f,w,n,p,w,o,e,w,v,l
6,e,f,s,w,f,n,f,w,b,h,...,s,w,w,p,w,o,e,k,a,g
7,e,f,y,n,t,n,f,c,b,p,...,s,g,w,p,w,o,p,n,y,d
8,p,f,f,y,f,f,f,c,b,h,...,k,b,b,p,w,o,l,h,y,p
9,p,k,y,n,f,y,f,c,n,b,...,k,p,p,p,w,o,e,w,v,p


In [564]:
validation_data = pd.read_table("validation.txt", sep=",", names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                                                  "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                                                  "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                                                  "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                                                  "veil-color","ring-number","ring-type","spore-print-color",
                                                  "population","habitat"])

validation_data.loc[validation_data['label'] == 'y', 'label'] = 'p'
validation_data.loc[validation_data['label'] == 'n', 'label'] = 'e'
validation_data

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,f,f,y,f,f,f,c,b,g,...,k,b,b,p,w,o,l,h,y,g
1,e,f,y,n,f,f,f,c,n,b,...,s,p,p,p,w,o,e,w,v,l
2,e,f,f,g,t,n,f,c,b,u,...,s,w,w,p,w,o,p,n,v,d
3,e,x,y,e,f,y,f,c,n,b,...,s,p,p,p,w,o,e,w,v,d
4,e,f,y,y,f,f,f,c,b,h,...,k,p,n,p,w,o,l,h,y,g
5,e,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,p
6,e,x,y,g,t,n,f,c,b,u,...,s,g,w,p,w,o,p,k,v,d
7,e,f,s,g,t,f,f,c,b,h,...,f,w,w,p,w,o,p,h,s,g
8,e,x,y,y,t,a,f,c,b,w,...,s,w,w,p,w,o,p,k,n,g
9,e,f,f,n,t,n,f,c,b,w,...,s,p,w,p,w,o,p,n,y,d


In [537]:
count_correct = 0
count_false = 0
for i,r in testing_data.iterrows():
    if r["label"] == classify(r,id3_root_entropy):
        count_correct = count_correct + 1
    else:
        count_false = count_false + 1
print("correct: ", count_correct," false: ",count_false)
    

correct:  2031  false:  0


In [563]:
count_correct = 0
count_false = 0
for i,r in validation_data.iterrows():
    if r["label"] == classify(r,id3_root_entropy):
        count_correct = count_correct + 1
    else:
        count_false = count_false + 1
print("correct: ", count_correct," false: ",count_false)
    

correct:  1014  false:  1017
